### Detector Systematics 
- Load the DetVar samples, applies the nue selection on them
- The after selection pickl is used to study the variations

-- efficieny at different stages

-- in function of a variable (plot)

In [2]:
import nue_selection_helper as nue_helper
from helpers import helpfunction as helper
import glob
import pandas as pd
import numpy as np

In [2]:
write_pickle = False
outfile = './input/Jun2020/lite/sys_after_training.pckl'
input_dir = "./input/Jun2020/syst/"
output_dir = "./output/"

In [3]:
samples = ['_'.join(f.split('/')[-1].split('.')[0].split('_')[:-1]) for f in glob.glob(input_dir+'*.pckl')]

In [4]:
if write_pickle:
    nue_helper.CreateAfterTraining(samples, input_dir, one_file=outfile)
data = pd.read_pickle(outfile)

In [5]:
data.keys()

dict_keys(['nue_DetVar_WireModAngleYZ', 'nue_DetVar_LYRayleigh', 'nue_DetVar_LYAttenuation', 'nue_DetVar_wiremod_ScaleYZ', 'nue_DetVar_LYDown', 'nue_DetVar_wiremod_ScaledEdX', 'nue_DetVar_WireModAngleXZ', 'nue_DetVar_CV', 'nue_DetVar_wiremod_ScaleX'])

In [6]:
# We grouped by 'event' was this a smart choice?
for sample in samples:
    assert len(np.unique(data[sample]['daughters'].xs(0, level='daughter').index)) == len(data[sample]['daughters'].xs(0, level='daughter').index)

In [27]:
for sample in samples:
    data[sample]['daughters']['nueccinc_new'] = data[sample]['daughters']['category'].isin([1,10,11])
    num_w = data[sample]['daughters'].eval('nueccinc*weightSplineTimesTune').xs(0, level='daughter')
    den_w = data[sample]['mc']['weightSplineTimesTune'][data[sample]['mc']['nueccinc']]
    print('{:<30}\t Efficiency: {:0.1%}+-{:0.1%}'.format(sample, *helper.effErr(num_w, den_w) ))

nue_DetVar_WireModAngleYZ     	 Efficiency: 84.2%+-0.2%
nue_DetVar_LYRayleigh         	 Efficiency: 84.1%+-0.2%
nue_DetVar_LYAttenuation      	 Efficiency: 83.8%+-0.2%
nue_DetVar_wiremod_ScaleYZ    	 Efficiency: 83.8%+-0.2%
nue_DetVar_LYDown             	 Efficiency: 84.6%+-0.2%
nue_DetVar_wiremod_ScaledEdX  	 Efficiency: 83.6%+-0.2%
nue_DetVar_WireModAngleXZ     	 Efficiency: 84.0%+-0.2%
nue_DetVar_CV                 	 Efficiency: 84.1%+-0.2%
nue_DetVar_wiremod_ScaleX     	 Efficiency: 83.8%+-0.2%
